In [ ]:
import threading
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageTk
import re
import jellyfish
from typing import List, Set
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AdamW
import torch
from bs4 import BeautifulSoup

In [19]:
stop_event = threading.Event()
# Kiểu mô hình mặc định
default_model = "PhoBERT"
model = None
tokenizer = None
word2vec_model = None
doc2vec_model = None
test1_path = None
test2_path = None
# độ dài mã thông báo
MAX_LENGTH = 512
local_model_dir = "./models"
matrix_file_path1 = None
matrix_file_path2 = None

In [20]:
df = pd.read_csv('D:/Python/Quora/Quora/quora-question-pairs/Data/Dataset/train/dataset.csv')

In [21]:
new_df = df

In [22]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,answer
0,0,1,2,Tháng 8 còn trường nào xét học bạ không?,Những trường Đại Học còn nhận học bạ trong thá...,1,"Có, Tháng 8 các trường Đại Học vẫn còn nhận"
1,1,3,4,Chính sách hỗ trợ học phí cho sinh viên sư phạ...,Những điều cần biết về chính sách hỗ trợ học p...,1,Theo Nghị định 116
2,2,5,6,Trường ĐHSP có cho thiếu học bạ trong hồ sơ nh...,Có thể bổ sung học bạ sau khi nộp hồ sơ vào ĐH...,1,Có thể bổ sung sau khi nộp hồ sơ
3,3,7,8,Chương trình học ngành SP Nga có phải 100% tiế...,Xuyên suốt bài giảng ngành SP Nga có phải 100%...,1,"Không, bài giảng ngành Sư phạm Nga tại trường ..."
4,4,9,10,Không thuộc đối tượng ưu tiên có thể đăng kí ở...,Điều kiện đăng ký ở ký túc xá của trường?,1,Cần đăng ký qua hệ thống trực tuyến hoặc theo ...


In [23]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    contractions = { 
    "sp": "sư phạm",
    "sn": "sinh năm",
    "bn": "bao nhiêu",
    "ngta": "người ta",
    "lm": "làm",
    "khg": "không",
    "ko": "không",
    "hok": "không",
    "khum": "không",
    "kg": "không",
    "đc": "được",
    "dc": "được",
    "đg": "đang",
    "ng": "người",
    "bt": "biết",
    "h": "giờ",
    "hc": "học",
    "vt": "viết",
    "vc": "việc",
    "trc": "trước",
    "j": "gì",
    "xg": "xong",
    "bx": "bữa",
    "vg": "vâng",
    "v": "vậy",
    "m": "mày",
    "t": "tôi",
    "s": "sao",
    "r": "rồi",
    "chs": "chơi",
    "ae": "anh em",
    "a": "anh",
    "e": "em",
    "cj": "chị",
    "đhsp": "đại học sư phạm",
    "đh": "đại học",
    "đhspkt": "đại học sư phạm kỹ thuật",
    "ktx": "kí túc xá",
    "đt": "điện thoại",
    "cntt": "công nghệ thông tin",
    "tp": "thành phố",
    "hcm": "hồ chí minh",
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q

In [24]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

c:\Users\Tienn Vo\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [25]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,answer
0,0,1,2,tháng 8 còn trường nào xét học bạ không,những trường đại học còn nhận học bạ trong thá...,1,"Có, Tháng 8 các trường Đại Học vẫn còn nhận"
1,1,3,4,chính sách hỗ trợ học phí cho sinh viên sư phạ...,những điều cần biết về chính sách hỗ trợ học p...,1,Theo Nghị định 116
2,2,5,6,trường đại học sư phạm có cho thiếu học bạ tro...,có thể bổ sung học bạ sau khi nộp hồ sơ vào đạ...,1,Có thể bổ sung sau khi nộp hồ sơ
3,3,7,8,chương trình học ngành sư phạm nga có phải 100...,xuyên suốt bài giảng ngành sư phạm nga có phải...,1,"Không, bài giảng ngành Sư phạm Nga tại trường ..."
4,4,9,10,không thuộc đối tượng ưu tiên có thể đăng kí ở...,điều kiện đăng ký ở ký túc xá của trường,1,Cần đăng ký qua hệ thống trực tuyến hoặc theo ...


In [26]:
def ensure_dir_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [27]:
ensure_dir_exists(local_model_dir)

In [28]:
def load_or_download_phobert():
    phobert_model_path = os.path.join(local_model_dir, "models--vinai--phobert-base")
    if not os.path.exists(phobert_model_path):
        print("Downloading PhoBERT model...")
        AutoModel.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)
        AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)
    else:
        print("PhoBERT model loaded from local storage.")
    return AutoModel.from_pretrained(
        "vinai/phobert-base", cache_dir=local_model_dir
    ), AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)

In [29]:
def preload_models():
    global model, tokenizer
    try:
        if not os.path.exists(local_model_dir):
            os.makedirs(local_model_dir)
            print(f"Directory created: {local_model_dir}")
        ensure_dir_exists(local_model_dir)
        model, tokenizer = load_or_download_phobert()
        print(f"Models loaded successfully.")
        time.sleep(1)
    except Exception as e:
        print(f"Error loading models: {str(e)}")

In [30]:
# Khai báo biến toàn cục
data_file_path1 = ""
data_file_path2 = ""
keywords_file_path = ""
matrix_tab3 = []

In [31]:
preload_models()

PhoBERT model loaded from local storage.
Models loaded successfully.


In [32]:
train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)

In [33]:
def preprocess_data(new_df):
    inputs = tokenizer(
        new_df['question1'].tolist(),
        new_df['question2'].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    inputs['labels'] = new_df['is_duplicate'].tolist()
    return inputs

In [34]:
# Áp dụng tiền xử lý
train_data = preprocess_data(train_df)
test_data = preprocess_data(test_df)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [35]:
class QuestionPairDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [36]:
# Tạo Dataset
train_dataset = QuestionPairDataset(train_data, train_df['is_duplicate'].tolist())
test_dataset = QuestionPairDataset(test_data, test_df['is_duplicate'].tolist())

In [37]:
# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [38]:
# Load mô hình PhoBERT với đầu ra là phân loại nhị phân
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)
model_embed = AutoModel.from_pretrained("vinai/phobert-base")
# Thiết lập optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Tienn Vo\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
if torch.cuda.is_available():
    print("GPU is available and being used.")
else:
    print("GPU is not available, using CPU instead.")

GPU is available and being used.


In [40]:
# Huấn luyện mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):  # Số epoch
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

Epoch 1, Loss: 0.4671964400806106
Epoch 2, Loss: 0.35414217700465367
Epoch 3, Loss: 0.2765344667057387
Epoch 4, Loss: 0.19841188358334014
Epoch 5, Loss: 0.13823983293946068


In [41]:
#đánh giá mô hình
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8117


In [42]:
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import numpy as np

# Tính độ chính xác tổng
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.2%}")  # In dạng phần trăm

# Lấy các chỉ số precision, recall, f1 cho từng class
report = classification_report(true_labels, predictions, target_names=["Negative", "Positive"], digits=2, output_dict=True)

# Tính số mẫu và số dự đoán đúng
num_samples = len(true_labels)
correct_preds = np.array(predictions) == np.array(true_labels)
num_correct = np.sum(correct_preds)

num_negative = sum(np.array(true_labels) == 0)
num_positive = sum(np.array(true_labels) == 1)
correct_negative = sum((np.array(true_labels) == 0) & (np.array(predictions) == 0))
correct_positive = sum((np.array(true_labels) == 1) & (np.array(predictions) == 1))


Accuracy: 81.17%


In [43]:
print("Kết quả đánh giá chi tiết:")
print("-" * 60)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("-" * 60)
print(f"{'Metric':<12}{'Negative':>12}{'Positive':>12}{'Avg/Total':>12}")
print("-" * 60)

precision_neg = report['Negative']['precision'] * 100
precision_pos = report['Positive']['precision'] * 100
recall_neg = report['Negative']['recall'] * 100
recall_pos = report['Positive']['recall'] * 100
f1_neg = report['Negative']['f1-score'] * 100
f1_pos = report['Positive']['f1-score'] * 100
precision_avg = report['weighted avg']['precision'] * 100
recall_avg = report['weighted avg']['recall'] * 100
f1_avg = report['weighted avg']['f1-score'] * 100

print(f"{'Precision':<12}{precision_neg:>11.2f}%{precision_pos:>12.2f}%{precision_avg:>12.2f}%")
print(f"{'Recall':<12}{recall_neg:>11.2f}%{recall_pos:>12.2f}%{recall_avg:>12.2f}%")
print(f"{'F1-score':<12}{f1_neg:>11.2f}%{f1_pos:>12.2f}%{f1_avg:>12.2f}%")
print("-" * 60)
print(f"{'Số mẫu':<12}{num_negative:>12}{num_positive:>12}{num_samples:>12}")
print(f"{'Dự đoán đúng':<12}{correct_negative:>12}{correct_positive:>12}{num_correct:>12}")


Kết quả đánh giá chi tiết:
------------------------------------------------------------
Accuracy: 81.17%
------------------------------------------------------------
Metric          Negative    Positive   Avg/Total
------------------------------------------------------------
Precision         89.24%       71.57%       82.43%
Recall            78.89%       84.82%       81.17%
F1-score          83.75%       77.63%       81.39%
------------------------------------------------------------
Số mẫu              3837        2404        6241
Dự đoán đúng        3027        2039        5066


In [44]:
model.save_pretrained("./phobert-finetuned")
tokenizer.save_pretrained("./phobert-finetuned")
model_embed.save_pretrained("./phobert-embed")

In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import faiss

# Tải lại mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./phobert-finetuned")
tokenizer = AutoTokenizer.from_pretrained("./phobert-finetuned")
model_embed = AutoModel.from_pretrained("./phobert-embed")
# Load FAISS index
index = faiss.read_index("faiss_index.bin")

# Dự đoán độ tương đồng
def predict_similarity(question1, question2):
    inputs = tokenizer(question1, question2, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    return probs[0][1].item()  # Xác suất thuộc lớp tương đồng (1)

In [46]:
def find_answer(input_question, dataset, threshold=0.6):
    # Tính độ tương đồng giữa câu hỏi đầu vào và các câu hỏi trong bộ dữ liệu
    max_similarity = -1
    best_match = None

    for index, row in dataset.iterrows():
        # Tính độ tương đồng giữa input_question và question1
        similarity1 = predict_similarity(input_question, row['question1'])
        # Tính độ tương đồng giữa input_question và question2
        similarity2 = predict_similarity(input_question, row['question2'])
        # Lấy độ tương đồng cao nhất
        max_current_similarity = max(similarity1, similarity2)

        # Nếu độ tương đồng cao hơn ngưỡng và cao hơn giá trị hiện tại
        if max_current_similarity > threshold and max_current_similarity > max_similarity:
            max_similarity = max_current_similarity
            best_match = row

    # Nếu tìm thấy câu hỏi trùng khớp, trả về câu trả lời
    if best_match is not None:
        return best_match['answer']
    else:
        return "Không tìm thấy câu trả lời phù hợp."

In [52]:
import faiss
def encode(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model_embed(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()  # Chuyển về 1D

# Tạo danh sách vector từ các câu hỏi
question_vectors = np.array([encode(row['question1']) for _, row in df.iterrows()])

# Kiểm tra kích thước
print(question_vectors.shape)  # Kết quả phải là (số lượng câu, 768)

# Khởi tạo FAISS Index
index = faiss.IndexFlatL2(question_vectors.shape[1])
index.add(question_vectors)  # Không còn lỗi

KeyboardInterrupt: 

In [53]:
# Hàm chuẩn hóa vector
def normalize(vec):
    return vec / np.linalg.norm(vec)

# Chuẩn hóa toàn bộ câu hỏi trước khi đưa vào FAISS
question_vectors = np.array([normalize(encode(row['question1'])) for _, row in df.iterrows()])

# Tạo FAISS index sử dụng Inner Product (dot product)
index = faiss.IndexFlatIP(question_vectors.shape[1])
index.add(question_vectors)
# Lưu FAISS index
faiss.write_index(index, "faiss_index.bin")

# Lưu các câu hỏi dưới dạng numpy để khôi phục sau này
np.save("questions.npy", df[['question1', 'question2', 'answer']].to_numpy(), allow_pickle=True)

In [54]:
def find_answer_faiss(input_question, df):
    input_vector = normalize(encode(input_question)).astype('float32').reshape(1, -1)
    D, I = index.search(input_vector, 1)  # Top-1
    best_match = df.iloc[I[0][0]]
    return best_match['answer'] if D[0][0] > 0.6 else "Không tìm thấy câu trả lời phù hợp."

In [59]:
def find_answer_faiss_all(input_question, df, threshold=0.6, top_k=5):
    input_vector = normalize(encode(input_question)).astype('float32').reshape(1, -1)
    # Tìm top_k câu hỏi gần nhất theo cosine similarity
    D, I = index.search(input_vector, top_k)
    # Tìm câu có độ tương đồng cao nhất trong top_k và > threshold
    best_idx = None
    best_score = -1

    for score, idx in zip(D[0], I[0]):
        if score > threshold and score > best_score:
            best_score = score
            best_idx = idx

    # Trả lại kết quả nếu tìm thấy
    if best_idx is not None:
        return df.iloc[best_idx]['answer']
    else:
        return "Không tìm thấy câu trả lời phù hợp."


In [55]:
# Ví dụ
question1 = "Trang Facebook nào nhất ở Ấn Độ?"
question2 = "Khi nào Facebook ra mắt ở Ấn Độ?"

In [60]:
test_df = new_df.head(20)
test_df['similarity_score'] = test_df.apply(lambda row: predict_similarity(row['question1'], row['question2']), axis=1)
test_df['answer_question'] = test_df.apply(lambda row: find_answer_faiss_all(row['question1'], df), axis=1)
test_df

C:\Users\Tienn Vo\AppData\Local\Temp\ipykernel_25152\3196361293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['similarity_score'] = test_df.apply(lambda row: predict_similarity(row['question1'], row['question2']), axis=1)
C:\Users\Tienn Vo\AppData\Local\Temp\ipykernel_25152\3196361293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['answer_question'] = test_df.apply(lambda row: find_answer_faiss_all(row['question1'], df), axis=1)


,id,qid1,qid2,question1,question2,is_duplicate,answer,similarity_score,answer_question
0,0,1,2,tháng 8 còn trường nào xét học bạ không,những trường đại học còn nhận học bạ trong thá...,1,"Có, Tháng 8 các trường Đại Học vẫn còn nhận",0.989326,"Có, Tháng 8 các trường Đại Học vẫn còn nhận"
1,1,3,4,chính sách hỗ trợ học phí cho sinh viên sư phạ...,những điều cần biết về chính sách hỗ trợ học p...,1,Theo Nghị định 116,0.992619,Theo Nghị định 116
2,2,5,6,trường đại học sư phạm có cho thiếu học bạ tro...,có thể bổ sung học bạ sau khi nộp hồ sơ vào đạ...,1,Có thể bổ sung sau khi nộp hồ sơ,0.991209,Có thể bổ sung sau khi nộp hồ sơ
3,3,7,8,chương trình học ngành sư phạm nga có phải 100...,xuyên suốt bài giảng ngành sư phạm nga có phải...,1,"Không, bài giảng ngành Sư phạm Nga tại trường ...",0.995262,"Không, bài giảng ngành Sư phạm Nga tại trường ..."
4,4,9,10,không thuộc đối tượng ưu tiên có thể đăng kí ở...,điều kiện đăng ký ở ký túc xá của trường,1,Cần đăng ký qua hệ thống trực tuyến hoặc theo ...,0.996000,Cần đăng ký qua hệ thống trực tuyến hoặc theo ...
5,5,11,12,năm nhất ngành sư phạm sinh học học những gì,chương trình học năm đầu tiên ngành sư phạm si...,1,Các môn nhập môn và các môn học phần chung,0.996866,Các môn nhập môn và các môn học phần chung
6,6,13,14,tìm trọ giá rẻ gần trường đại học sư phạm hcm,tìm trọ giá rẻ gần cơ sở chính đhsphcm,1,Tìm trên các trang thông tin trọ cho sinh viên,0.005309,Tìm trên các trang thông tin trọ cho sinh viên
7,7,15,16,khi nào đăng kí ký túc xá,thời gian bắt đầu đăng kí ký túc xá là lúc nào,1,Thường vào đầu mỗi năm học hoặc theo thông báo...,0.989269,Thường vào đầu mỗi năm học hoặc theo thông báo...
8,8,17,18,chia sẻ kinh nghiệm ở ký túc xá,review ký túc xá trường,1,"Ký túc xá ở quận 11, cơ sở vật chất tiện nghi,...",0.992990,"Ký túc xá ở quận 11, cơ sở vật chất tiện nghi,..."
9,9,19,20,anh chị cho em hỏi ngày nhập học của trường,khi nào thì trường bắt đầu nhập học,1,Bắt đầu vào cuối tháng 8,0.996809,Bắt đầu vào cuối tháng 8
